In [ ]:
%reset -f

In [ ]:
import numpy as np
import pandas as pd
import datetime
import time
import re
import requests
from itertools import islice

api_key = pd.read_json('fmp_api_key.json').iloc[0].to_list()[0]
data_path = '/Users/ayemyatwinshwe/repos/data/trading_data/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15'}

In [ ]:
# Earning call transcripts
def get_earning_calls(ticker, year):
    """
    Inputs: stock ticker and year
    Outputs: pandas dataframe for earning call transcripts
    """
    response = requests.get("https://financialmodelingprep.com/api/v4/batch_earning_call_transcript/" +
                       ticker + "?year="+ str(year) + "&apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
        
    return df

In [ ]:
df_tickers = pd.read_csv(data_path + 'tickers.csv')
tickers_all = df_tickers['symbol'].values
len(tickers_all)

In [ ]:
# break up the list tickers into smaller lists of ~ 500 elements
length_to_split = [500]*10 + [541]
iter_tickers = iter(tickers_all)
tickers_lists = [list(islice(iter_tickers, elem)) for elem in length_to_split]

In [ ]:
def get_mul_earning_calls(tickers, years):
    
    earning_calls = pd.concat([get_earning_calls(tickers[0], year) for year in years])
    
    for ticker in tickers[1:]:
        earning_calls_new = pd.concat([get_earning_calls(ticker, year) for year in years])
        earning_calls = pd.concat([earning_calls, earning_calls_new])
    
    return earning_calls.reset_index(drop=True)

In [ ]:
years = [2019, 2020, 2021]
temp = 8

for tickers in tickers_lists[8:]:
    
    temp = temp+1
    earning_calls = get_mul_earning_calls(tickers, years)
    earning_calls.to_parquet(data_path + 'earning_calls_' + str(temp), engine='pyarrow', compression='gzip')  
    time.sleep(5)

In [ ]:
df_1 = pd.read_parquet(data_path + 'earning_calls_1')
df_2 = pd.read_parquet(data_path + 'earning_calls_2')
df_3 = pd.read_parquet(data_path + 'earning_calls_3')
df_4 = pd.read_parquet(data_path + 'earning_calls_4')
df_5 = pd.read_parquet(data_path + 'earning_calls_5')
df_6 = pd.read_parquet(data_path + 'earning_calls_6')
df_7 = pd.read_parquet(data_path + 'earning_calls_7')
df_8 = pd.read_parquet(data_path + 'earning_calls_8')
df_9 = pd.read_parquet(data_path + 'earning_calls_9')
df_10 = pd.read_parquet(data_path + 'earning_calls_10')
df_11 = pd.read_parquet(data_path + 'earning_calls_11')

In [ ]:
earning_calls = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11])

In [ ]:
earning_calls = earning_calls.convert_dtypes()

In [ ]:
earning_calls.head()

In [ ]:
earning_calls['date'] = pd.to_datetime(earning_calls['date'])

In [ ]:
earning_calls.to_parquet(data_path + 'earning_calls', engine='pyarrow', compression='gzip')  

In [ ]:
earning_calls['date'].max()